# ROS Navigation

## Chapter 4: Path Planning

<img src="img/husky_sim.png" width="1000"></img>

<p style="background:green;color:white;">SUMMARY</p>

Estimated time of completion: <b>3 hours</b><br><br>What you will learn with this unit?
<ul>
  <li>What means Path Planning in ROS Navigation?</li>
  <li>How does Path Planning work?</li>
  <li>How does the move_base node work?</li>
  <li>What is a Costmap?</li>
</ul>

<p style="background:green;color:white;">END OF SUMMARY</p>

We're arriving to the end of the Course, guys! For now, we've seen how to create a Map of an environment, and how to localize the robot in it. So, at this point (and assuming everything went well), we have all we need in order to perform Navigation. That is, we're now ready to plan trajectories in order to move the robot from a pose A to a pose B.
<br><br>
In this Chapter you'll learn how does the Path Planning process work in ROS, and all the elements that take place in it. But first, as we've been doing in previous Chapters, let's have a look at our digital best friend, RViz.

## Visualize Path Planning in Rviz

As you've already seen in previous chapters, you can also launch RViz and add displays in order to watch the Path Planning process of the robot. For this chapter, you'll basically need to use 3 elements of RViz:
<ul>
<li>Map Display (Costmaps)</li>
<li>Path Displays (Plans)</li>
<li>2D Tools</li>
</ul>

<p style="background:#EE9023;color:white;">Exercise 1.1</p>
<br>
a) Execute the next command in order to launch the move_base node.<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p><br>
roslaunch husky_navigation move_base_demo.launch<br>
</th>
</tr>
</table>
<br>

b) Open the Graphic Interface and execute the following command in order to start RViz.<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p><br>
rosrun rviz rviz<br>
</th>
</tr>
</table>
<br>

c) Properly congifure RViz in order to visualize the necessary parts.<br>

#### Visualize Costmaps

<ol>
<li>Click the Add button under Displays and chose the Map element.</li> 
<li>Set the topic to <b>/move_base/global_costmap/costmap</b> in order to visualize the global costmap</li>
<li>Change the topic to <b>/move_base/local_costmap/costmap</b> in order to visualize the local costmap.</li>
<li>You can also have 2 Map displays, one for each costmap.</li>
</ol>

#### Visualize Plans

<ol>
<li>Click the Add button under Displays and chose the Path element.</li> 
<li>Set the topic to <b>/move_base/NavfnROS/plan</b> in order to visualize the global plan.</li> 
<li>Change the topic to <b>/move_base/DWAPlannerROS/local_plan</b> in order to visualize the local plan.</li>
<li>You can also have 2 Path displays, one for each plan.</li>
</ol>

d) Use the 2D Pose Estimate tool in order to provide an initial pose of the robot.
<br>
<img src="img/2d_pose_estimate_rviz.png" width="200"></img>

e) Use the 2D Nav Goal tool in order to send a goal pose to the robot. Make sure to select an unoccupied (dark grey) or unexpected (light grey) location.
<br>
<img style="" src="img/2d_nav_goal_rviz.png" width="200"></img>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.1</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span>Bear in mind that, until you don't set a 2D Nav Goal, the planning process won't start. This means that until that, you won't be able to visualize any plan in RViz.<br>
<span style="color:orange">Note 1: </span>In order to the 2D tools to work, the Fixed Frame at Rviz must be set to map.<br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.1</p>

Global Costmap:

<img src="img/rviz_global_costmap.png" width="500"></img>

Local Costmap:

<img src="img/rviz_local_costmap.png" width="500"></img>

Global Plan:

<img src="img/rviz_planning1.png" width="500"></img>

Local Plan (in red) and Global Plan (in green):

<img src="img/global_vs_local.png" width="500"></img>

<p style="background:#EE9023;color:white;">End of Exercise 1.1</p>

That's awesome, right? But what has just happened? What was that 2D Nav Goal tool I used in order to move the robot? And what's a Costmap? And how does ROS calculate the trajectories?
<br><br>Keep calm!! We'll get into it in just a moment. But first, let's refresh and introduce some new elements, so you can have a better understanding of the whole process.

## The move_base package

The move_base package contains the <b>move_base node</b>. Doesn't it sound familiar to you? Well it should, since you were introduced to it in the Basic Concepts Chapter! The move_base node is one of the major elements in the ROS Navigation Stack, since it links all the elements that take place in the Navigation process. Let's say it's like the Architect in Matrix, or the Force in Star Wars. Without this node, the ROS Navigation Stack wouldn't make any sense!

Ok! We've understood that the move_base node is very important, but... what it is exactly? What it does? Great question!

The <b>main function of the move_base node is to move the robot from its current position to a goal position</b>. Basically, this node is an implementation of a <i>SimpleActionServer</i>, which takes a goal pose with message type <i>geometry_msgs/PoseStamped</i>. Therefore, we can send position goals to this node by using a <i>SimpleActionClient</i>. 

This Action Server subscribes to the topic move_base_node/goal, which is the input of the Navigation Stack, in order to get the position goal.

<p style="background:#EE9023;color:white;">Exercise 1.2</p>
<br>
a) In a Web Shell, visualize the <i>move_base/goal</i> topic.
<br><br>
b) As you did in the previous exercise, send a goal to the robot by using the 2D Nav Goal tool in RViz.
<br><br>
c) Check what happens in the topic you are listening.

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.2</p>

<img src="img/send_goal_msg.png" width="600"></img>

<p style="background:#EE9023;color:white;">End of Exercise 1.2</p>

As you've seen, when you send a goal to the robot through RViz, a new message is published into the /move_base/goal topic.

When this node receives a goal pose, it links to components such as the <b>global_planner, local_planner, recovery_behavior, global_costmap and local_costmap</b>, and generates an output, which is a velocity command with the message type <i>geometry_msgs/Twist</i>, and sends it to the <b>/cmd_vel</b> topic in order to move the robot.

OK, but... what are all those parts the move_base node links to? Let's have a quick look at them:
<ul>
<li><b>global-planner</b>: This package is in charge of planning the optimum path for the robot in order to go from the initial position to a goal position. It takes the data of an static map and plans a trajectory.</li>
<li><b>local-planner</b>: This package is in charge of navigating the robot through a section of the global plan. It takes data from odometry  and from the sensors, and sends velocity commands to the robot in order to perform a section of the global plan.</li>
<li><b>rotate-recovery</b>: This package helps the robot in case it finds an obstacle and gets stuck. It will perform a 360º degree rotation in order to find a free path to move.</li>
<li><b>clear-costmap-recovery</b>: This package also helps the robot in case it gets stuck. It will clear the current costmap, and change it a costmap based on the static map.
</li>
<li><b>costmap-2D</b>: This package is in charge of building costmaps. It can build costmaps based on the sensors and odometry data, and also based on a static map. 
</li>
</ul>

But that's not all! The move_base node also interfaces with other packages:
<ul>
<li><b>map-server</b>: This package allows you to save the maps created by the costmap-2D package.</li>
<li><b>AMCL</b>: As you already know, this package allows the robot to localize itself in the map.</li>
<li><b>gmapping</b>: As you already know, this packages allows you to create 2D maps, based on the sensors and odometry data.</li>
</ul>

This ones are more familiar to you, right?

So, summarizing:

After getting the current position of the robot, we can send a goal position to the <b>move_base</b> node. This node will then send this goal position to a <b>global planner</b> which will plan a path from the current robot position to the goal position. This plan is with respect to the <b>global costmap</b>, which is feeding from the <b>map server</b>.
<br><br>
The <b>global planner</b> will then send this path to the <b>local planner</b>, which executes each segment of the global plan. The <b>local planner</b> gets the odometry and the laser data values and finds a collision free local plan for the robot. The <b>local planner</b> is associated with the <b>local costmap</b>, which can monitor the obstacle(s) around the robot.
<br><br>
The <b>global and local costmap</b> are tied with the laser scan data. If the robot is stuck somewhere, the recovery behavior nodes, such as the <b>clear costmap recovery</b> or <b>rotate recovery</b>, will be called.
Finally, the <b>local planner</b> generates the velocity commands and sends them to the base controller. The robot base controller will then convert this commands into real robot movement.

Wow!! That sounds amazing!! Ok, so now that we have a better picture of the whole thing, let's see what really happened in the previous exercise.
<br>
Bascially, you sent a goal pose to the <i>move_base_node/goal</i> topic using the 2D Nav Goal tool of Rviz. Then, the move_base node received this goal pose, and and started the whole process described just above. Now everything makes more sense, right?

Let's now have a look at the different topics and services that take place in this process.

### Topics

The Action Server of the move_base node uses, as every action server, these 5 topics:
<ul>
<li><b>move_base/goal (move_base_msgs/MoveBaseActionGoal)</b></li>
<li><b>move_base/cancel (actionlib_msgs/GoalID)</b></li>
<li><b>move_base/feedback (move_base_msgs/MoveBaseActionFeedback)</b></li>
<li><b>move_base/status (actionlib_msgs/GoalStatusArray)</b></li>
<li><b>move_base/result (move_base_msgs/MoveBaseActionResult)</b></li>
</ul>

<p style="background:#EE9023;color:white;">Exercise 1.2</p>
<br>
Without using Rviz, send a pose goal to the move_base node.
<br><br>
a) Use the command line tool in order to send this goal to the Action Server of the move_base node.
<br><br>
b) Visualize through the web shells all the topics involved in the action, and check its output while the action is taking place, and when it's done.
<br>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.2</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span>Remember that the SimpleAcionServer subscribes to the /move_base_node/goal topic in order to read the pose goal.</i><br>
<span style="color:orange">Note 2: </span>In order to see an example of a valid message for the /move_base/goal topic, you can listen to the topic while you send a pose goal via the 2D Nav Goal tool of RViz.<br>
<span style="color:orange">Note 3: </span>Keep in mind that, in order to be able to send goals to the Action Server, the move_base node must be launched.<br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.2</p>

Sending goal:

<img src="img/send_goal_msg.png" width="600"></img>

Echo feedback:

<img src="img/echo_feedback.png" width="600"></img>

Echo status accepted:

<img src="img/status_accepted.png" width="600"></img>

Echo status reached:

<img src="img/status_reached.png" width="600"></img>

Echo result:

<img src="img/echo_result.png" width="600"></img>

<p style="background:#EE9023;color:white;">End of Exercise 1.2</p>

<p style="background:#EE9023;color:white;">Exercise 1.3</p>
<br>
a) Create a new package named <b>send_goals</b>. Add rospy as a dependency.
<br><br>
b) Inside this package, create a file named <b>send_goal_client.py</b>. Write into this file the code for an Action Client in order to send messages to the Action Server of the move_base node.
<br><br>
c) Using this Action Client, move the robot to three different Poses of the Map. When the robot has reached the 3 poses, start over again creating a loop, so the robot will keep going to these 3 poses over and over.
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.3</p>

<p style="background:#EE9023;color:white;">End of Exercise 1.3</p>

### Services

Besides the Action, the move_base node also provides some Services:
<ul>
<li><b>make_plan (nav_msgs/GetPlan)</b>: This service sllows you to ask for a plan to a given pose without causing move_base to execute that plan.</li>
<li><b>clear_unknown_space (std_srvs/Empty)</b>: This service allows you to clear unknown space in the area directly around the robot. This is useful when move_base has its costmaps stopped for a long period of time and then is started again in a new location in the environment.</li>
<li><b>clear_costmaps (std_srvs/Empty)</b>: This service allows you to clear obstacles in the costmaps used by move_base. This could cause a robot to hit things and should be used with caution.</li>
</ul>

<p style="background:#EE9023;color:white;">Exercise 1.4</p>
<br>
Create a Service Client that will call one of the Services introduced above in order to get the Plan to a given Pose, without causing the robot to move.
<br><br>
a) Create a new package named <b>make_plan</b>. Add rospy as a dependency.
<br><br>
b) Inside this package, create a file named <b>make_plan_caller.py</b>. Write into this file the code for your Action Client.

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.4</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span>The type of message used by the /make_plan service is nav_msgs/GetPlan.</i><br>
<span style="color:orange">Note 2: </span>When filling this message in order to call the service, you don't have to fill all the fields of the message. Check the following message example:
<br>
<img src="img/calling_service.png" width="600"></img>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.4</p>

Returned Plan:

<img src="img/service_plan.png" width="600"></img>

<p style="background:#EE9023;color:white;">End of Exercise 1.4</p>

### Parameters

The move_base node also has some parameters that you can modify in order to change its behavior. This parameters are set in a YAML file that is loaded in the move_base launch file. You can have a look at a complete list with all the parameters that you can use to customize the move_base node here: http://wiki.ros.org/move_base. 
<br><br>
Now, let's check some of the most important ones:

<ul>
<li><b>base_global_planner (default: "navfn/NavfnROS")</b>: Determines the global planner plugin to use.</li>
<li><b>base_local_planner (default: "base_local_planner/TrajectoryPlannerROS")</b>: Determines the local planner plugin to use.</li>
<li><b>recovery_behaviors (default: [{name: conservative_reset, type: clear_costmap_recovery/ClearCostmapRecovery}, {name: rotate_recovery, type: rotate_recovery/RotateRecovery}, {name: aggressive_reset, type: clear_costmap_recovery/ClearCostmapRecovery}])</b>: Determines the recovery behavior plugins to use. If the robot can't execute a given plan, it will perform these recovery behaviors.</li>
<li><b>controller_frequency (default: 20.0)</b>: The frequency at which the node will send velocity commands to the robot.</li>
<li><b>controller_patience (default: 15.0)</b>: The time (in seconds) that the controller will wait without receiving a valid plan before it starts to perform recovery behaviors.</li>
<li><b>planner_frequency (default: 0.0)</b>: The frequency at which the global planner will calculate a new path. If this value is 0.0, the global planner will only run when a new goal is received or when the local planner can't execute its path.</li>
<li><b>planner_patience (default: 5.0)</b>: The time (in seconds) that the planner will try to find a valid plan until it starts to perform recovery behaviors.</li>
<li><b>recovery_behavior_enabled (default: true)</b>: It enables or disables the recovery behaviors.</li>
<li><b>oscillation_timeout (default: 0.0)</b>: The time (in seconds) that the robot is allowed to oscillate before performing recovery behaviors. A value of 0.0 corresponds to an infinite timeout.</li>
<li><b>oscillation_distance (default: 0.5)</b>: How far in meters the robot must move to be considered not to be oscillating.</li>
<li><b>shutdown_costmaps (default: false)</b>: Determines whether or not to shutdown the costmaps of the node when move_base is in an inactive state.</li>
<li><b>max_planning_retries (default: -1)</b>: How many times to allow for planning retries before performing recovery behaviors. If this value is -1.0, it allows for infinite retries.</li>
</ul>

As you already know, when launching the node, this parameters are loaded into the ROS Parameter Server. This means, you can list them by using the following command:

<img src="img/rosparam_list.png" width="500"></img>

And you can access them by using the command:

Next you have an image of some of the parameters listed:

<img src="img/rosparam_get.png" width="800"></img>

Here you have an example move_base_params.yaml file:

In [ ]:
# Move base node parameters.

shutdown_costmaps: false

controller_frequency: 5.0
controller_patience: 3.0

planner_frequency: 1.0
planner_patience: 5.0

oscillation_timeout: 10.0
oscillation_distance: 0.2

# local planner - default is trajectory rollout
base_local_planner: "dwa_local_planner/DWAPlannerROS"

base_global_planner: "navfn/NavfnROS" #alternatives: global_planner/GlobalPlanner, carrot_planner/CarrotPlanner

recovery_behavior_enabled: true

recovery_behaviors:
  - name: 'super_conservative_reset1'
    type: 'clear_costmap_recovery/ClearCostmapRecovery'
  - name: 'conservative_reset1'
    type: 'clear_costmap_recovery/ClearCostmapRecovery'
  - name: 'aggressive_reset1'
    type: 'clear_costmap_recovery/ClearCostmapRecovery'
  - name: 'clearing_rotation1'
    type: 'rotate_recovery/RotateRecovery'

super_conservative_reset1:
  reset_distance: 3.0
conservative_reset1:
  reset_distance: 1.5
aggressive_reset1:
  reset_distance: 0.0
clearing_rotation1:
  sim_granularity: 0.017
  frequency: 20.0

<p style="background:#EE9023;color:white;">Exercise 1.5</p>
<br>
<p style="color:red;"><b>IMPORTANT: Before starting with this Exercise, make sure you've stopped the previously launched move_base node by pressing Ctrl + C on the console where you executed the command.</b></p>
<br>
In the move_base parameters file, change the <i>base_global_planner</i> used.
<br><br>
Check what happens now.

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.5</p>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.5</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span><br>
<span style="color:orange">Note 2: </span><br>
</th>
</tr>
</table>

<p style="background:#EE9023;color:white;">End of Exercise 1.5</p>

By the way... these "costmaps" sound important to the whole Planning Process. What are they? That's a very good question, my friend!

## The Costmap 2-D Package

This package uses sensor data and information from the static map (the Map we created in Chapter 2) in order to build a 2D or 3D occupancy grid, and inflate costs in a 2D costmap based on the occupancy grid and a user specified inflation radius.

Ok, but... what does Costmap mean? What is it? That's again a great question! A Costmap is the data structure that represents places that are safe for the robot to be in a grid of cells. Usually, the values in the costmap are binary, representing free space or places where the robot would be in collision. 

Next you have an example of how a costmap would like:

<img src="img/costmap_example.png" width="500"></img>

### Costmap Types

There are the two types of costmaps:
<ul>
<li><b>global costmap</b></li>
<li><b>local costmap</b></li>
</ul>

Our robot will move through the map using 2 types of navigation: <b>global navigation and local navigation</b>. 
<br>
The global navigation is used to create paths for a goal in the map or a far-off distance. The <b>global costmap is used for the global navigation</b>. This is the one we will be discussing this Chapter. 
<br>
The local navigation is used to create paths in the nearby distances and avoid obstacles. The <b>local costmap is used for the local navigation</b>. This one will be discussed in the next Chapter (Obstacle Avoidance).

Basically, the difference between the global and local costmap is that the global planner uses the global costmap to generate a long-term plan while the local planner uses the local costmap to generate a short-term plan.

<p style="background:#EE9023;color:white;">Exercise 1.6</p>
<br>
a) Open Rviz and add the proper displays in order to visualize the Global and the Local Costmaps.
<br><br>
b) Execute the following command in order to spawn an obstacle in the room.<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p><br>
rosrun gazebo_ros spawn_model -file /home/user/object.urdf -urdf -x 0 -y 0 -z 1 -model my_object<br>
</th>
</tr>
</table>

c) Launch the keyboard Teleop and move near the spawned object.
<br>
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #3</p><br>
roslaunch husky_teleop keyboard_teleop.launch<br>
</th>
</tr>
</table>

d) Check the differences between the Global and the Local Costmap

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.6</p>

Husky facing the spawned obstacle:

<img src="img/facing_obstacle.png" width="600"></img>

Global Costmap (Obstacle doesn't appears):

<img src="img/global_costmap_obstacle.png" width="600"></img>

Local Costmap (Obstacle does appear):

<img src="img/local_costmap_obstacle.png" width="600"></img>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.6</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span>You can use the command <i>roslaunch husky_rviz_launchers view_planning.launch</i> in order to launch RViz with predefined displays.<br>
</th>
</tr>
</table>

<p style="background:#EE9023;color:white;">End of Exercise 1.6</p>

### Costmap Values

Each cell in the costmap has an integer value in the range {0,255}. There are some special values frequently used in this range, which work as follows:
<ul>
<li><b>255 (NO_INFORMATION)</b>: Reserved for cells where not enoguh information is known.</li>
<li><b>254 (LETHAL_OBSTACLE</b>: Indicates a collision causing obstacle was sensed in this cell</li>
<li><b>253 (INSCRIBED_INFLATED_OBSTACLE)</b>: Indicates no obstacle, but moving the center of the robot to this location will result in a collision</li>
<li><b>0 (FREE_SPACE)</b>: Cells where there are no obstacles and therefore, moving the center of the robot to this position will not result in a collision</li>
</ul>

### Parameters

Configuration of the costmaps consists of three files where we can set up different parameters:

<ol>
<li><b>global_costmap_params.yaml</b>: Parameters related to the global costmap</li>
<li><b>local_costmap_params.yaml</b>: Parameters related to the local costmap</li>
<li><b>costmap_common_params.yaml</b>: Parameters related to both the global and local costmaps</li>
</ol>

In order to make things easier, we can divide these 3 files in 2 types of parameters:
<ul>
<li><b>Common parameters</b>: These parameters will be the same for both the local and the global costmaps. Therefore, they will be setted in the <i>costmap_common_params.yaml</i> file.</li>
<li><b>Specific parameters</b>: These parameters will vary for the local and the global costmap. Therefore, they will be setted in the <i>global_costmap_params.yaml</i> and the <i>local_costmap_params.yaml</i> files.</li>
</ul>

<p style="background:#EE9023;color:white;">Exercise 1.7</p>
<br>
a) Localize these files in your system.
<br><br>
b) Open them using a text editor tool and check how they are build.

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.7</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span>Bear in mind that the names of the files may slightly vary depending on who creates them.<br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.7</p>

Common params:

<img src="img/common_params.png" width="500"></img>

Global costmap params:

<img src="img/global_costmap_params.png" width="500"></img>

Local costmp_params:

<img src="img/local_costmap_params.png" width="500"></img>

<p style="background:#EE9023;color:white;">End of Exercise 1.7</p>

### Layers

In order to simplify (and clarify) the configuration of costmaps, ROS uses layers. Layers are like "blocks" of parameters that are related between them. For instance, the <b>static map, the sensed obstacles and the inflated areas are separated into different layers</b>. These layers are definde in the <i>common_costmap_parameters.yaml</i> file, and then added to the <i>local_costmap_params.yaml</i> and <i>global_costmap_params.yaml</i> files.

To add a layer to a configuration file of a costmap, you will specify it in the plugins area. Have a look at the following line:

Here, you're adding to your costmap configuration an static map layer. You can add as many layers as you want:

For instance, check out the following local costmap parameters file:

In [ ]:
local_costmap:
   global_frame: odom
   robot_base_frame: /base_footprint
   update_frequency: 5.0
   publish_frequency: 2.0
   static_map: false
   rolling_window: true
   width: 4.0
   height: 4.0
   resolution: 0.05
   transform_tolerance: 0.5
   plugins:
    - {name: obstacle_layer,      type: "costmap_2d::VoxelLayer"}
    - {name: inflation_layer,     type: "costmap_2d::InflationLayer"}

You can see how at the end of the file we've added the obstacle and the inflation layers.

Now, have a look at the following common costmap paremeters file:

In [ ]:
max_obstacle_height: 0.60  # assume something like an arm is mounted on top of the robot

# Obstacle Cost Shaping (http://wiki.ros.org/costmap_2d/hydro/inflation)
robot_radius: 0.20  # distance a circular robot should be clear of the obstacle (kobuki: 0.18)
# footprint: [[x0, y0], [x1, y1], ... [xn, yn]]  # if the robot is not circular

map_type: voxel

obstacle_layer:
  enabled:              true
  max_obstacle_height:  0.6
  origin_z:             0.0
  z_resolution:         0.2
  z_voxels:             2
  unknown_threshold:    15
  mark_threshold:       0
  combination_method:   1
  track_unknown_space:  true    #true needed for disabling global path planning through unknown space
  obstacle_range: 2.5
  raytrace_range: 3.0
  origin_z: 0.0
  z_resolution: 0.2
  z_voxels: 2
  publish_voxel_map: false
  observation_sources:  scan bump
  scan:
    data_type: LaserScan
    topic: scan
    marking: true
    clearing: true
    min_obstacle_height: 0.25
    max_obstacle_height: 0.35
  bump:
    data_type: PointCloud2
    topic: mobile_base/sensors/bumper_pointcloud
    marking: true
    clearing: false
    min_obstacle_height: 0.0
    max_obstacle_height: 0.15
  # for debugging only, let's you see the entire voxel grid
  
#cost_scaling_factor and inflation_radius were now moved to the inflation_layer ns
inflation_layer:
  enabled:              true
  cost_scaling_factor:  5.0  # exponential rate at which the obstacle cost drops off (default: 10)
  inflation_radius:     0.5  # max. distance from an obstacle at which costs are incurred for planning paths.

static_layer:
  enabled:              true

Here you can see how are defined the different parameters of the layers:

### Map Initialization

There are 2 main ways to initialize a costmap:
<ol>
<li>By setting the <b>static_map</b> parameter to true, you'll fill the costmap with a <b>user-generated</b> static map (as the one we created in the Mapping Unit). In this case, the costmap is initialized to match the width, height and obstacle information provided by the static map. This configuration is normally used in conjunction with a localization system, such as amcl. This is the method you'll use to initialize a <b>global costmap</b>.</li>
<li>Define a width and a height and set the <b>rolling_window</b> parameter to true. This parameter keeps the robot in the center of the costmap as it moves throughout the environment, dropping obstacle information from the map as the robot moves. This type of configuration is most often used in an odometric coordinate frame, where the robot only cares about obstacles within a local area. This is the method you'll use to initialize a <b>local costmap</b>.</li>
</ol>

These parameters are usually different for global and local costmaps, so they can be considered specific parameters.

<p style="background:#EE9023;color:white;">Exercise 1.8</p>
<br>
a) In the local costmap parameters file, change the initialization of the map to use a static map.
<br><br>
b) Repeat again the Exercise 1.6, and see what happens now.<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.8</p>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.8</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span><br>
<span style="color:orange">Note 2: </span><br>
</th>
</tr>
</table>

<p style="background:#EE9023;color:white;">End of Exercise 1.8</p>

### Map Updates

The costmap performs map update cycles at a rate specified by the <b>update_frequency</b> parameter. Each cycle works as follows:
<ol>
<li>Sensor data comes in</li>
<li>Marking and clearing operations are performed</li>
<li>The appropied cost values are assigned as described above</li>
<li>obstacle inflation is performed on each cell with a LETHAL_OBSTACLE value. This consists of propagating cost values outwards from each occupied cell out to a specified inflation radius.</li>
</ol>

The update frequency parameter is a specific parameter, so it's set in both the local and the global costmap parameters files.

<p style="background:#EE9023;color:white;">Exercise 1.9</p>
<br>
a) In the local and global costmap parameters files, change the <b>update frequency</b> parameter of the map to be slower.
<br><br>
b) Repeat again the Exercise 1.6, and see what happens now.<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.9</p>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.9</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span><br>
<span style="color:orange">Note 2: </span><br>
</th>
</tr>
</table>

<p style="background:#EE9023;color:white;">End of Exercise 1.9</p>

### Marking and Clearing

The costmap automatically subscribes to the sensor topics and updates itself according to the data it receives from them. Each sensor is used to either mark (insert obstacle information into the costmap), clear (remove obstacle information from the costmap), or both.
<br><br> 
A marking operation is just an index into an array to change the cost of a cell.<br>
A clearing operation, however, cosists of raytracing through a grid from the origin of the sensor outwards for each observation reported.
<br><br>
The marking and clearing operations can be defined in the obstacle layer. For instance, some of the parameters you can define are:
<ul>
<li><b>/max_obstacle_height (double, default: 2.0)</b></li>
<li><b>/obstacle_range (double, default: 2.5)</b></li>
<li><b>/raytrace_range (double, default: 3.0)</b></li>
<li><b>/origin_z (double, default: 0.0)</b></li>
<li><b>/z_resolution (double, default: 0.2)</b></li>
<li><b>/z_voxels (int, default: 10)</b></li>
<li><b>/unknown_threshold (int, default: ~name/z_voxels)</b></li>
<li><b>/mark_threshold (int, default: 0)</b></li>
<li><b>/publish_voxel_map (bool, default: false)</b></li>
<li><b>/observation_sources (string, default: "")</b></li>
</ul>
<br>
Each source_name in observation_sources defines a namespace in which parameters can be set:
<ul>
<li><b>/source_name/topic (string, default: source_name)</b></li>
<li><b>/source_name/data_type (string, default: "PointCloud")</b></li>
<li><b>/source_name/clearing (bool, default: false)</b></li>
<li><b>/source_name/marking (bool, default: true)</b></li>
<li><b>/source_name/min_obstacle_height (double, default: 0.0)</b></li>
<li><b>/source_name/max_obstacle_height (double, default: 2.0)</b></li>
</ul>

These are common parameters, so they will be set on the common parameters file. Check the layers section for an example file.

<p style="background:#EE9023;color:white;">Exercise 2.0</p>
<br>
a) In the common parameters files, change the <b>obstacle range</b> parameter to be higher.
<br><br>
b) Repeat again the Exercise 1.6, and see what happens now.<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.0</p>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 2.0</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span><br>
<span style="color:orange">Note 2: </span><br>
</th>
</tr>
</table>

<p style="background:#EE9023;color:white;">End of Exercise 2.0</p>

### Inflation

Inflation is the process of propagating cost values out from occupied cells that decrease with distance. For this purpose, there are 5 specific symbols defined for costmap values:
<ul>
<li><b>"Lethal"</b> cost means that there is an actual obstacle in a cell. So if the robot center were in that cell, it would obviously be in collision.</li>
<li><b>"Inscribed"</b> cost means that a cell is less that the robot's inscribed radius away from an obstacle. So the robot would be in collision with an obstacle if the robot center is in a cell with this cost. </li>
<li><b>"Possibly Circumscribed"</b> cost is similar to inscrived, but using the robot's circumscribed radius as cutoff distance. Thus, if the robot center lies in a cell at or above this value, then it depends on the orientation of the robot whether it collides with an obstacle or not. </li>
<li><b>"Freespace"</b> cost is assumed to be zero, and it means that there is nothing that should keep the robot from going there. </li>
<li><b>"Unknown"</b> cost means there is no information about a given cell. The user of the costmap can interpret this as the want. </li>
</ul>

All other costs are assigned a value between "Freespace" and "Possibly Circumscribed" depending on their distance from a "Lethal" cell and the decay function provided by the user.

Inflation parameters can be defined in the inflation layer. These are the parameters you can define:
<ul>
<li><b>/inflation_radius (double, default: 0.55)</b></li>
<li><b>/cost_scaling_factor (double, default: 10.0)</b></li>
</ul>

<p style="background:#EE9023;color:white;">Exercise 2.1</p>
<br>
a) In the common parameters files, change the <b>inflation radius</b> parameter to be higher.
<br><br>
b) Repeat again the Exercise 1.6, and see what happens now.<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.1</p>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 2.1</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span><br>
<span style="color:orange">Note 2: </span><br>
</th>
</tr>
</table>

<p style="background:#EE9023;color:white;">End of Exercise 2.1</p>

### Topics involved

<ul>
<li><b>/grid (nav_msgs/OccupancyGrid)</b>: The values in the costmap</li>
<li><b>/grid_updates (map_msgs/OccupancyGridUpdate)</b>: The value of the updated area of the costmap</li>
<li><b>/voxel_grid (costmap_2d/VoxelGrid)</b>: Optionally advertised when the underlying occupancy grid uses voxels and the user requests the voxel grid be published.</li>
</ul>

<p style="background:#EE9023;color:white;">Exercise 2.2</p>
<br>
Listen to the topic where the costmap node publishes it's values and try to localize the different areas of the map.
<br>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 2.2</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span><br>
<span style="color:orange">Note 2: </span><br>
<span style="color:orange">Note 2: </span><br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.2</p>

<p style="background:#EE9023;color:white;">End of Exercise 2.2</p>

Now you've understood better what are costmaps and how they work, it's time to get deeper into the path planning processes. Let's go!

## The Global Planner Package

This package contains the <b>global_planner node</b>, which <b>is in charge of calculating a safe path in order to arrive to the goal pose</b>. This planned path doesn't take in account the readings that the robot sensors are doing while moving. It just uses the global costmap information in order to plan the path. When it's done, it publishes the path into the <i>/plan</i> topic.

The global plan is computed before the robot starts moving towards the destination.

The global planner assumes a circular robot and operates on a costmap to find a minimum cost plan from a start point to an end point in a grid.

The path can be computed using different planning algorithms, depending on how you set up the parameters.

### Standard Behavior

<img src="img/GlobalPlanner.png" width="500"></img>

### Grid Path

<img src="img/GridPath.png" width="500"></img>

### Simple Potential Calculation

<img src="img/Nonquad.png" width="500"></img>

### A*

<img src="img/AStar2.png" width="500"></img>

### Dijkstra

<img src="img/Dijkstra.png" width="500"></img>

### Old Navfn

<img src="img/OldNavFn.png" width="500"></img>

<p style="background:#EE9023;color:white;">Exercise 2.3</p>
<br>
Open Rviz and a add a Display in order to be able to see the Global Plan.
<br><br>
Subscribe to the topic where the Global Planner publishes its planned path, and have a look at it.

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 2.3</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span><br>
<span style="color:orange">Note 2: </span><br>
<span style="color:orange">Note 3: </span><br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.3</p>

<img src="img/planned_path.png" width="300"></img>

<p style="background:#EE9023;color:white;">End of Exercise 2.3</p>

As always, you can configure this node in order to customize its behaviour. The parameters for the global planner are placed in a file called <b>global_planner_params.yaml</b>. Let's see some of this parameters:
<ul>
<li><b>/allow_unknown (bool, default: true)</b>: Specifies whether or not to allow the planner to create plans that traverse unknown space.</li>
<li><b>/default_tolerance (double, default: 0.0)</b>: A tolerance on the goal point for the planner. The planner will attempt to create a plan that is as close to the specified goal as possible but no further than default_tolerance away.</li>
<li><b>/visualize_potential (bool, default: false)</b>: Specifies whether or not to visualize the potential area computed via a PointCloud2.</li>
<li><b>/use_dijkstra (bool, default: true)</b>: If true, use dijkstra's algorithm. Otherwise, A*.</li>
<li><b>/use_quadratic (bool, default: true)</b>: If true, use the quadratic approximation of the potential. Otherwise, use a simpler calculation.</li>
<li><b>/use_grid_path (bool, default: false)</b>: If true, create a path that follows the grid boundaries. Otherwise, use a gradient descent method.</li>
<li><b>/old_navfn_behavior (bool, default: false)</b>: If for some reason, you want global_planner to exactly mirror the behavior of navfn, set this to true (and use the defaults for the other boolean parameters)</li>
</ul>

Here you can see an example of a global planner parameters file:

In [ ]:
GlobalPlanner:                                  # Also see: http://wiki.ros.org/global_planner
  old_navfn_behavior: false                     # Exactly mirror behavior of navfn, use defaults for other boolean parameters, default false
  use_quadratic: true                           # Use the quadratic approximation of the potential. Otherwise, use a simpler calculation, default true
  use_dijkstra: true                            # Use dijkstra's algorithm. Otherwise, A*, default true
  use_grid_path: false                          # Create a path that follows the grid boundaries. Otherwise, use a gradient descent method, default false

  allow_unknown: true                           # Allow planner to plan through unknown space, default true
                                                #Needs to have track_unknown_space: true in the obstacle / voxel layer (in costmap_commons_param) to work
  default_tolerance: 0.0                        # If goal in obstacle, plan to the closest point in radius default_tolerance, default 0.0

  visualize_potential: false                   

<p style="background:#EE9023;color:white;">Exercise 2.4</p>
<br>
Change the <b>use_dijkstra</b> parameter to false, and repeat Exercise 2.3. Check if something changes now.
<br>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 2.4</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span><br>
<span style="color:orange">Note 2: </span><br>
<span style="color:orange">Note 2: </span><br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.4</p>

<p style="background:#EE9023;color:white;">End of Exercise 2.4</p>

## The Base Local Planner Package

This package provides a controller that drives a mobile base in the environment. Given a plan to follow (provided by the global-planner) and a costmap (provided by costmap-2D), it provides velocity commands in order to move the robot. Unlike the global planner, the local planner monitors the odometry and the laser data, and chooses a collision free local plan (let's imagine the local plan as a smaller part of the global plan) for the robot.
<br><br>
The base local planner can recompute the robot's path on the fly in order to keep the robot from striking objects, yet still allowing it to reach its destination.

Along the way, the planner creates, at least locally around the robot, a value function, represented as a grid map. This value function encodes the costs of traversing through the grid cells. The controller's job is to use this value function to determine which velocities it has to send to the robot.

Summarizing, The basic idea of how this controller works is as follows:
<ol>
<li>Discretely sample in the robot's control space (dx,dy,dtheta)</li>
<li>For each sampled velocity, perform forward simulation from the robot's current state to predict what would happen if the sampled velocity were applied for some (short) period of time.</li>
<li>Evaluate (score) each trajectory resulting from the forward simulation, using a metric that incorporates characteristics such as: proximity to obstacles, proximity to the goal, proximity to the global path, and speed. Discard illegal trajectories (those that collide with obstacles).</li>
<li>Pick the highest-scoring trajectory and send the associated velocity to the mobile base.</li>
<li>Rinse and repeat.</li>
</ol>

### Topics involved

<ul>
<li><b>/global_plan (nav_msgs/Path)</b>: The portion of the global plan that the local planner is currently attempting to follow. Used primarily for visualization purposes.</li>
<li><b>/local_plan (nav_msgs/Path)</b>: The local plan or trajectory that scored the highest on the last cycle. Used primarily for visualization purposes.</li>
<li><b>/cost_cloud (sensor_msgs/PointCloud2)</b>: The cost grid used for planning. Used for visualization purposes. See the publish_cost_grid_pc parameter for enabling/disabling this visualization.</li>
</ul>

<p style="background:#EE9023;color:white;">Exercise 2.5</p>
<br>
Open Rviz and add Displays in order to be able to visualize the /global_plan and the /local_plan topics of the Local Planner.
<br><br>
Send a Goal Pose to the Robot and visualize both topics.
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.5</p>

<img src="img/dwa_local_vs_global.png" width="800"></img>

<p style="background:#EE9023;color:white;">End of Exercise 2.5</p>

### Algorithms

The base local planner implements the <b>Trajectory Rollout and the Dynamic Window algorithms</b> in order to calculate its paths.

#### Trajectory Rollout Algorithm

This algorithm works as follows:
<ol>
<li>Discretely sample in the robot's control space</li>
<li>For each sampled velocity, performs forward simulation from the robot's current state to predict what would happen if the sampled velocity was applied.</li>
<li>Evaluate each trajectory resulting from the forward simulation.</li>
<li>Discard illegal trajectories.</li>
<li>Pick the highest-scoring trajectory and send the associated velocities to the mobile base.</li>
<li>Rinse and Repeat.</li>
</ol>

#### Dynamic Window ALgorithm (DWA)

DWA differs from Trajectory Rollout in how the robot's space is sampled. Trajectory Rollout samples from the set of achievable velocities over the entire forward simulation period given the acceleration limits of the robot, while DWA samples from the set of achievable velocities for just one simulation step given the acceleration limits of the robot.
<br><br>
DWA is a more efficient algorithm because it samples a smaller space, but may be outperformed by TRajectory Rollout for robots with low acceleration limits because DWA does not forward simulate constant accelerations.
<br><br>
In practice, DWA and Trajectory Rollout perform similar, so it's recommended to use DWA because of its efficiency gains.

<p style="background:#EE9023;color:white;">Exercise 2.6</p>
<br>
Change the <b>dwa</b> parameter in the local planner parameters file to false in order to use the Trajectory Rollout Algorithm.

<br>
Check if you notice some differences in the performance.
<br>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 2.6</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span><br>
<span style="color:orange">Note 2: </span><br>
<span style="color:orange">Note 2: </span><br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.6</p>

<p style="background:#EE9023;color:white;">End of Exercise 2.6</p>

### Parameters

As always, The base local planner has many parameters in order to customize its behavior. Let's check some of this parameters:

#### Robot Configuration Parameters

<ul>
<li><b>/acc_lim_x (double, default: 2.5)</b></li>
<li><b>/acc_lim_y (double, default: 2.5)</b></li>
<li><b>/acc_lim_theta (double, default: 3.2)</b></li>
<li><b>/max_vel_x (double, default: 0.5)</b></li>
<li><b>/min_vel_x (double, default: 0.1)</b></li>
<li><b>/max_vel_theta (double, default: 1.0)</b></li>
<li><b>/min_vel_theta (double, default: -1.0)</b></li>
<li><b>/min_in_place_vel_theta (double, default: 0.4)</b></li>
<li><b>/escape_vel (double, default: -0.1)</b></li>
<li><b>/holonomic_robot (bool, default: true)</b></li>
</ul>

#### Goal Tolerance Parameters

<ul>
<li><b>/yaw_goal_tolerance (double, default: 0.05)</b></li>
<li><b>/xy_goal_tolerance (double, default: 0.10)</b></li>
<li><b>/latch_xy_goal_tolerance (bool, default: false)</b></li>
</ul>

#### Forward Simulation Parameters

<ul>
<li><b>/sim_time (double, default: 1.0)</b></li>
<li><b>/sim_granularity (double, default: 0.025)</b></li>
<li><b>/angular_sim_granularity (double, default: ~name/sim_granularity)</b></li>
<li><b>/vx_samples (integer, default: 3)</b></li>
<li><b>/vtheta_samples (integer, default: 20)</b></li>
<li><b>/controller_frequency (double, default: 20.0)</b></li>
</ul>

#### Trajectory Scoring Parameters

<ul>
<li><b>/meter_scoring (bool, default: false)</b></li>
<li><b>/pdist_scale (double, default: 0.6)</b></li>
<li><b>/gdist_scale (double, default: 0.8)</b></li>
<li><b>/occdist_scale (double, default: 0.01)</b></li>
<li><b>/dwa (bool, default: true)</b></li>
</ul>

#### Oscillation Prevention Parameters

<ul>
<li><b>/oscillation_reset_dist (double, default: 0.05)</b></li>
</ul>

#### Global Plan Parameters

<ul>
<li><b>/prune_plan (bool, default: true)</b></li>
</ul>

<p style="background:#EE9023;color:white;">Exercise 2.7</p>
<br>
Execute the command used in the Exercise 1.1, but modify some of this parameters in order to see how they affect to the Local Path Planning.
<br>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 2.7</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.7</p>

<p style="background:#EE9023;color:white;">End of Exercise 2.7</p>

### Dynamic Reconfigure

Until now we've seen how to change parameters by modifying them in the parameters files. But, guess what... this is not the only way you can change parameters! You can also change dynamic parameters by using the rqt_reconfigure tool. Follow the next steps:

<p style="background:#EE9023;color:white;">Exercise 2.7</p>
<br>
a) Run the next command in order to open the rqt_reconfigure tool.
<br>
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p><br>
rosrun rqt_reconfigure rqt_reconfigure<br>
</th>
</tr>
</table>

b) Open the move_base group.
<br><br>
c)Select the DWAPlannerROS node.
<br><br>
d) Set the <b>pdist_scale</b> parameter to something high like 2.5.
<br><br>
e) Open Rviz and visualize the Global and Local Plans.

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 2.7</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.7</p>

<img src="img/rqt_reconfigure.png" width="600"></img>

<p style="background:#EE9023;color:white;">End of Exercise 2.7</p>

## The DWA Local Planner Package

This package provides an implementation of the Dynamic Window Approach to local robot navigation on a plane. It is basically a re-write of the base_local_planner's DWA (Dynamic Window Approach) option, but the code is a lot cleaner and easier to understand, particularly in the way that trajectories are simulated. So, for applications that use the DWA approach for local planning, the dwa_local_planner is probaly the best choice. 
<br><br>
This package supports any robot who's footprint can be represented as a convex polygon or cicrle, and exposes its configuration as ROS parameters that can be set in a launch file. The parameters for this planner are also dynamically reconfigurable.

### Parameters

As always, The dwa local planner has many parameters in order to customize its behavior. Let's check some of this parameters:

#### Robot Configuration Parameters

#### Goal Tolerance Parameters

#### Forward Simulation Parameters

#### Trajectory Scoring Parameters

#### Oscillation Prevention Parameters

#### Global Plan Parameters

Here you have an example of the dwa_local_planner_params.yaml:

In [ ]:
DWAPlannerROS:

# Robot Configuration Parameters - Kobuki
  max_vel_x: 0.5  # 0.55
  min_vel_x: 0.0

  max_vel_y: 0.0  # diff drive robot
  min_vel_y: 0.0  # diff drive robot

  max_trans_vel: 0.5 # choose slightly less than the base's capability
  min_trans_vel: 0.1  # this is the min trans velocity when there is negligible rotational velocity
  trans_stopped_vel: 0.1

  # Warning!
  #   do not set min_trans_vel to 0.0 otherwise dwa will always think translational velocities
  #   are non-negligible and small in place rotational velocities will be created.

  max_rot_vel: 5.0  # choose slightly less than the base's capability
  min_rot_vel: 0.4  # this is the min angular velocity when there is negligible translational velocity
  rot_stopped_vel: 0.4

  acc_lim_x: 1.0 # maximum is theoretically 2.0, but we
  acc_lim_theta: 2.0
  acc_lim_y: 0.0      # diff drive robot

# Goal Tolerance Parameters
  yaw_goal_tolerance: 0.3  # 0.05
  xy_goal_tolerance: 0.15  # 0.10
  # latch_xy_goal_tolerance: false

# Forward Simulation Parameters
  sim_time: 1.0       # 1.7
  vx_samples: 6       # 3
  vy_samples: 1       # diff drive robot, there is only one sample
  vtheta_samples: 20  # 20

# Trajectory Scoring Parameters
  path_distance_bias: 64.0      # 32.0   - weighting for how much it should stick to the global path plan
  goal_distance_bias: 24.0      # 24.0   - wighting for how much it should attempt to reach its goal
  occdist_scale: 0.5            # 0.01   - weighting for how much the controller should avoid obstacles
  forward_point_distance: 0.325 # 0.325  - how far along to place an additional scoring point
    stop_time_buffer: 0.2         # 0.2    - amount of time a robot must stop in before colliding for a valid traj.
  scaling_speed: 0.25           # 0.25   - absolute velocity at which to start scaling the robot's footprint
  max_scaling_factor: 0.2       # 0.2    - how much to scale the robot's footprint when at speed.

# Oscillation Prevention Parameters
  oscillation_reset_dist: 0.05  # 0.05   - how far to travel before resetting oscillation flags

# Debugging
  publish_traj_pc : true
  publish_cost_grid_pc: true
  global_frame_id: odom


# Differential-drive robot configuration - necessary?
#  holonomic_robot: false

<p style="background:#EE9023;color:white;">Exercise 2.8</p>
<br>
Execute the command used in the Exercise 1.1, but modify some of this parameters in order to see how they affect to the Local Path Planning.
<br>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 2.8</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span><br>
<span style="color:orange">Note 2: </span><br>
<span style="color:orange">Note 2: </span><br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.8</p>

<p style="background:#EE9023;color:white;">End of Exercise 2.8</p>

## The Rotate Recovery Package

This package provides a simple recovery behavior that attempts to clear out space by rotating the robot 360 degrees  if local obstacles are found in the way.

It has some parameters you can customize in order to change or improve its behavior:

### Rotate Recovery Parameters

<ul>
<li><b>/sim_granularity (double, default: 0.017)</b>: The distance in radians between checks for obstacles when checking if an in-place rotation is safe. Defaults to 1 degree.</li>
<li><b>/frequency (double, default: 20.0)</b>: The frequency in HZ at which to send velocity commands to the mobile base.</li>
</ul>

### TrajectoryPlanner Parameters

<p style="color:red;"> IMPORTANT: These parameters are already set when using the base_local_planner local planner, they only need to be set explicitly for the recovery behavior if a different local planner is used.</p>

<ul>
<li><b>/yaw_goal_tolerance (double, default: 0.05)</b>: The tolerance in radians for the controller in yaw/rotation when achieving its goal</li>
<li><b>/acc_lim_th (double, default: 3.2)</b>: The rotational acceleration limit of the robot in radians/sec^2</li>
<li><b>/max_rotational_vel (double, default: 1.0)</b>: The maximum rotational velocity allowed for the base in radians/sec</li>
<li><b>/min_in_place_rotational_vel (double, default: 0.4)</b>: The minimum rotational velocity allowed for the base while performing in-place rotations in radians/sec</li>
</ul>

These parameters are set in the move_base parameters file. Go back to the move_base section in order to refresh it.

<p style="background:#EE9023;color:white;">Exercise 2.9</p>
<br>
Force the robot to move to an obstacle and check if the Rotate Recovery behavior launches.
<br>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 2.9</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.9</p>

<p style="background:#EE9023;color:white;">End of Exercise 2.9</p>

## The Clear Costmap Recovery package

This package provides a simple recovery behavior that clears out space by clearing obstacles outside of a specified region from the robot's map.

The radius away from the robot (in meters) which obstacles will be removed from the costmap when they are reverted to the static map, can be setted by modifying the next parameter:
<ul>
<li><b>/reset_distance</b></li>
</ul>

This parameter is set in the move_base parameters file. Go back to the move_base section in order to refresh it.

<p style="background:#EE9023;color:white;">Exercise 3.0</p>
<br>
Force the robot to move to an obstacle and check if the Clear Costmap Recovery behavior launches.
<br>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 3.0</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 3.0</p><br>

<p style="background:#EE9023;color:white;">End of Exercise 3.0</p>

## Other Useful Visualizations in Rviz

Until now we've already seen some ways of visualizing different parts of the move_base node process throgu Rviz. But there are a couple more that may be interesting to know:

### Robot Footprint

It shows the footprint of the robot.

<img src="img/footprint.png" width="700"></img>

### Current Goal

To show the goal pose that the navigation stack is attempting to achieve add a Pose Display and set its topic to /move_base_simple/goal. You will now be able to see the goal pose as a red arrow. It can be used to know the final position of the robot.

<img src="img/current_goal.png" width="700"></img>

<p style="background:#EE9023;color:white;">Exercise 3.1</p>
<br>
Open Rviz and try to visualize the elements described above.
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 3.1</p>

<p style="background:#EE9023;color:white;">End of Exercise 3.1</p>

## SUMMARIZING AREA

<img src="img/overview_tf_small.png" width="800"></img>